In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import sys
sys.path.append('/gdrive/My Drive/Major/Code/Scraping Twitter Data')

In [ ]:
import private
import re
import tweepy
import difflib
import pandas as pd
import numpy as np
import pickle
import copy
import matplotlib.pyplot as plt
from datetime import date

In [ ]:
auth = tweepy.OAuthHandler("-","-")
auth.set_access_token("---","-")
api = tweepy.API(auth)

#Extracting Covid Tweets

## Twitter Scraped Features Description


![alt text](https://drive.google.com/uc?id=1LAjmD-RIR-oswQ0ocKaXK708Z7thsrAs)









In [ ]:
def get_list_of_user_mentions(tweet):
    user_handles = []

    try:
            
        mentions = tweet._json['entities']['user_mentions']
        for item in mentions:
            user_handles.append(item['screen_name'])
        return user_handles

    except:
        return user_handles

    

In [ ]:
def get_list_of_photos(tweet):
    photos_list = []

    try:
            
        photos = tweet._json['entities']['media']
        for item in photos:
            photos_list.append(item['media_url_https'])
        return photos_list
    
    except:
        return photos_list

In [ ]:
def get_replies_count(tweet,user_handle):
    
    try:
            
        replies=[] 
        q = 'to:'+user_handle
        for full_tweets in tweepy.Cursor(api.user_timeline,screen_name=user_handle,timeout=999999).items(10):
            for tweet in tweepy.Cursor(api.search,q, since_id=tweet_id, result_type='recent',tweet_mode='extended',timeout=999999).items(1000):
                if hasattr(tweet, 'in_reply_to_status_id_str'):
                    if (tweet.in_reply_to_status_id_str==full_tweets.id_str):
                        replies.append(tweet.full_text)

        #Removing Duplicate Replies
        replies = list(dict.fromkeys(replies))
        return len(replies)

    except:
        return 0

In [ ]:
def get_all_hashtags(tweet):
    hashtags_list = []

    hashtags = tweet._json['entities']['hashtags']
    for item in hashtags:
        hashtags_list.append(item['text'])


    return hashtags_list

In [ ]:
def get_retweet_id(tweet):
    try:
        retweet_id = tweet._json['retweeted_status']['id']
        return retweet_id
    except:
        return 0

In [ ]:
def get_retweet_date(tweet):
    try:
        retweet_date = tweet._json['retweeted_status']['created_at']
        return retweet_date
    except:
        return ""

In [ ]:
def get_retweet_user_id(tweet):
    try:
        retweet_user_id = tweet._json['retweeted_status']['user']['id']
        return retweet_user_id
    except:
        return ""

In [ ]:
def check_tweet_exists(test_tweet, texts_list):
    counter = 1
    test_tweet     = test_tweet.split()

    for existing_tweet in texts_list:
        existing_tweet = existing_tweet.split()
        n              = len(existing_tweet)
        m              = 0
        for word in test_tweet:
            if word in existing_tweet:
                m = m + 1
                if(m/n > 0.5):                                             ### If the new tweet is 50% similar with existing tweet then don't append it
                    #print("Helllo this tweet has been rejected") 
                    return True
        # print("This tweet has passed test number ",counter)
        counter = counter + 1
    return False



In [ ]:
### References --> https://towardsdatascience.com/selenium-tweepy-to-scrap-tweets-from-tweeter-and-analysing-sentiments-1804db3478ac

def get_tweets(query, desired_lang = 'hi',maxTweets = 10, tweetsPerQry=10): 
        # empty list to store parsed tweets 
        tweets = [] 
        texts_list = []
        sinceId = None
        max_id = -1
        tweetCount = 0
        
        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = api.search(q=query, lang= desired_lang, tweet_mode= 'extended',count=tweetsPerQry)
                    else:
                        new_tweets = api.search(q=query, lang = desired_lang, tweet_mode= 'extended',count=tweetsPerQry,
                                                since_id=sinceId)
                else:
                    if (not sinceId):
                        new_tweets = api.search(q=query, lang = desired_lang, tweet_mode= 'extended', count=tweetsPerQry,
                                                max_id=str(max_id - 1))
                    else:
                        new_tweets = api.search(q=query, lang = desired_lang, tweet_mode= 'extended', count=tweetsPerQry,
                                                max_id=str(max_id - 1),
                                                since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break
                    
                for tweet in new_tweets:

                    parsed_tweet = {} 

                    # Making Sure that the tweets don't have more than 50% word similarity

                    already_existing_tweet_flag = check_tweet_exists(tweet.full_text,texts_list)
                    if(already_existing_tweet_flag):
                        continue
                    else:
                        tweets.append(parsed_tweet)
                        texts_list.append(tweet.full_text)


                    parsed_tweet['id']                     = tweet.id
                    parsed_tweet['conversation_id']        = tweet.id_str
                    parsed_tweet['created_at']             = tweet.created_at
                    parsed_tweet['date']                   = str(tweet.created_at).split()[0]
                    parsed_tweet['time']                   = str(tweet.created_at).split()[1]
                    parsed_tweet['time_zone']              = "UTC" 
                    parsed_tweet['user_id']                = tweet.user.id
                    parsed_tweet['username']               = tweet.user.screen_name
                    parsed_tweet['name']                   = tweet.user.name
                    parsed_tweet['place']                  = tweet.user.location
                    parsed_tweet['tweet']                  = tweet.full_text
                    parsed_tweet['mentions']               = get_list_of_user_mentions(tweet)
                    parsed_tweet['urls']                   = tweet._json['entities']['urls']
                    parsed_tweet['photos']                 = get_list_of_photos(tweet)
                    parsed_tweet['replies_count']          = 0 #get_replies_count(tweet,tweet.user.screen_name)
                    parsed_tweet['retweet_count']          = tweet.retweet_count
                    parsed_tweet['likes_count']            = tweet.favorite_count
                    parsed_tweet['hashtags']               = get_all_hashtags(tweet)
                    parsed_tweet['cashtags']               = []
                    parsed_tweet['link']                   = "http://twitter.com/anyuser/status/" + tweet.id_str
                    parsed_tweet['retweet']                = tweet.retweeted
                    parsed_tweet['quote_url']              = " "
                    parsed_tweet['video']                  = 0
                    parsed_tweet['geo']                    = tweet._json['geo']
                    parsed_tweet['source']                 = tweet._json['source']
                    parsed_tweet['rt_user_id']             = get_retweet_user_id(tweet)
                    parsed_tweet['retweet_date']           = get_retweet_date(tweet)
                    parsed_tweet['retweet_id']             = get_retweet_id(tweet)
                                     
                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id

            except tweepy.TweepError as e:
                print("Tweepy error : " + str(e))
                break
        
        return pd.DataFrame(tweets)

In [ ]:
df = get_tweets("covid",'hi',maxTweets=30,tweetsPerQry=20)
df

Downloaded 20 tweets
Downloaded 40 tweets


,id,conversation_id,created_at,date,time,time_zone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweet_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,geo,source,rt_user_id,retweet_date,retweet_id
0,1393167720020025345,1393167720020025345,2021-05-14 11:34:02,2021-05-14,11:34:02,UTC,18186549,oneindiaHindi,Oneindia Hindi,Bangalore,"स्पूतनिक-वी वैक्सीन की कीमत का हो गया ऐलान, जा...",[],"[{'url': 'https://t.co/bXXL8mWDHC', 'expanded_...",[],0,0,0,"[SputnikVVaccine, SputnikVVaccineinIndia, Sput...",[],http://twitter.com/anyuser/status/139316772002...,False,,0,None,"<a href=""https://help.twitter.com/en/using-twi...",,,0
1,1393167702051614725,1393167702051614725,2021-05-14 11:33:58,2021-05-14,11:33:58,UTC,925254155131748353,RajeshY6762,Rajesh Yadav,ALLAHABAD,RT @kmsingh1963: श्रीनिवास @srinivasiyc Covid-...,"[kmsingh1963, srinivasiyc]",[],[],0,1,0,[],[],http://twitter.com/anyuser/status/139316770205...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",114089497,Fri May 14 11:33:23 +0000 2021,1393167553405493254
2,1393167695437205507,1393167695437205507,2021-05-14 11:33:57,2021-05-14,11:33:57,UTC,812960111610236928,bansalsonu95,Sonu Bansal,"Jaipur, India",RT @vinodkapri: मैं कभी कभी सोचता हूँ कि वो कौ...,[vinodkapri],[],[],0,2782,0,[],[],http://twitter.com/anyuser/status/139316769543...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",160588215,Fri May 14 07:32:48 +0000 2021,1393107009264701440
3,1393167680719310849,1393167680719310849,2021-05-14 11:33:53,2021-05-14,11:33:53,UTC,1271852296754864128,Nileshj30966877,Nilesh Jadav,,ab pata chala ki kyu @smritiirani ji aur @Mane...,"[smritiirani, Manekagandhibjp, BJP4India]","[{'url': 'https://t.co/i8DihmcB6Y', 'expanded_...",[],0,0,0,[],[],http://twitter.com/anyuser/status/139316768071...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",,,0
4,1393167658908934146,1393167658908934146,2021-05-14 11:33:48,2021-05-14,11:33:48,UTC,1387475468266311680,GNeemrot,Ganesh Neemrot,,RT @zeerajasthan_: कोरोना आपदा के कारण सारे त्...,[zeerajasthan_],"[{'url': 'https://t.co/Wii5mkZSWc', 'expanded_...",[],0,3,0,"[rajasthan, jhalwar, coronavirus]",[],http://twitter.com/anyuser/status/139316765890...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",1587915768,Fri May 14 11:00:06 +0000 2021,1393159180203171843
5,1393167639866925056,1393167639866925056,2021-05-14 11:33:43,2021-05-14,11:33:43,UTC,1372495337760436228,Awadh460,Awadh,Kingdom of Saudi Arabia,RT @Live_Hindustan: #RubinaDilaik ने #Eid के म...,[Live_Hindustan],"[{'url': 'https://t.co/B5L3ywz4bz', 'expanded_...",[],0,70,0,"[RubinaDilaik, Eid]",[],http://twitter.com/anyuser/status/139316763986...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",108852592,Fri May 14 10:09:54 +0000 2021,1393146544421961729
6,1393167634808537088,1393167634808537088,2021-05-14 11:33:42,2021-05-14,11:33:42,UTC,454609043,Amitkrkanhaiya,Amit Kumar Kanhaiya,Delhi,RT @ndtvindia: कोविड की राहत सामग्री को लेकर य...,[ndtvindia],[],[],0,253,0,[],[],http://twitter.com/anyuser/status/139316763480...,False,,0,None,"<a href=""http://twitter.com/download/iphone"" r...",321271735,Fri May 14 09:20:51 +0000 2021,1393134203663908864
7,1393167603049197574,1393167603049197574,2021-05-14 11:33:35,2021-05-14,11:33:35,UTC,959359483184115712,Bijendr90826388,Bijendra Singh,भारत,RT @thewirehindi: गोवा: शीर्ष सरकारी अस्पताल म...,[thewirehindi],"[{'url': 'https://t.co/9irscDaR8V', 'expanded_...",[],0,7,0,[Covid19],[],http://twitter.com/anyuser/status/139316760304...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",817340591046414336,Fri May 14 11:12:03 +0000 2021,1393162185879265280
8,1393167590315364354,1393167590315364354,2021-05-14 11:33:31,2021-05-14,11:33:31,UTC,979348471521034241,DhootNavratan,Navratan Dhoot,"Rajasthan, India",RT @MyGovHindi: मास्क हमेशा सही तरीके से पहनें...,[MyGovHindi],[],[],0,22,0,[],[],http://twitter.com/anyuser/status/1

In [ ]:
df.to_csv("/gdrive/My Drive/Major/Dataset/Scrapping tweets dem/covid_tweets_raw_features_dump.csv",index=False)

#Extracting User Features

## User Features

![alt text](https://drive.google.com/uc?id=1iXpdyY9eQw-HN7knBKdfhOhf4SPDqPlr)

In [ ]:
df = pd.read_csv("/gdrive/My Drive/Major/Dataset/Scrapping tweets dem/covid_tweets_raw_features_dump.csv")
df

,id,conversation_id,created_at,date,time,time_zone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweet_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,geo,source,rt_user_id,retweet_date,retweet_id
0,1393167720020025345,1393167720020025345,2021-05-14 11:34:02,2021-05-14,11:34:02,UTC,18186549,oneindiaHindi,Oneindia Hindi,Bangalore,"स्पूतनिक-वी वैक्सीन की कीमत का हो गया ऐलान, जा...",[],"[{'url': 'https://t.co/bXXL8mWDHC', 'expanded_...",[],0,0,0,"['SputnikVVaccine', 'SputnikVVaccineinIndia', ...",[],http://twitter.com/anyuser/status/139316772002...,False,,0,NaN,"<a href=""https://help.twitter.com/en/using-twi...",NaN,NaN,0
1,1393167702051614725,1393167702051614725,2021-05-14 11:33:58,2021-05-14,11:33:58,UTC,925254155131748353,RajeshY6762,Rajesh Yadav,ALLAHABAD,RT @kmsingh1963: श्रीनिवास @srinivasiyc Covid-...,"['kmsingh1963', 'srinivasiyc']",[],[],0,1,0,[],[],http://twitter.com/anyuser/status/139316770205...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",1.140895e+08,Fri May 14 11:33:23 +0000 2021,1393167553405493254
2,1393167695437205507,1393167695437205507,2021-05-14 11:33:57,2021-05-14,11:33:57,UTC,812960111610236928,bansalsonu95,Sonu Bansal,"Jaipur, India",RT @vinodkapri: मैं कभी कभी सोचता हूँ कि वो कौ...,['vinodkapri'],[],[],0,2782,0,[],[],http://twitter.com/anyuser/status/139316769543...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",1.605882e+08,Fri May 14 07:32:48 +0000 2021,1393107009264701440
3,1393167680719310849,1393167680719310849,2021-05-14 11:33:53,2021-05-14,11:33:53,UTC,1271852296754864128,Nileshj30966877,Nilesh Jadav,NaN,ab pata chala ki kyu @smritiirani ji aur @Mane...,"['smritiirani', 'Manekagandhibjp', 'BJP4India']","[{'url': 'https://t.co/i8DihmcB6Y', 'expanded_...",[],0,0,0,[],[],http://twitter.com/anyuser/status/139316768071...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,0
4,1393167658908934146,1393167658908934146,2021-05-14 11:33:48,2021-05-14,11:33:48,UTC,1387475468266311680,GNeemrot,Ganesh Neemrot,NaN,RT @zeerajasthan_: कोरोना आपदा के कारण सारे त्...,['zeerajasthan_'],"[{'url': 'https://t.co/Wii5mkZSWc', 'expanded_...",[],0,3,0,"['rajasthan', 'jhalwar', 'coronavirus']",[],http://twitter.com/anyuser/status/139316765890...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",1.587916e+09,Fri May 14 11:00:06 +0000 2021,1393159180203171843
5,1393167639866925056,1393167639866925056,2021-05-14 11:33:43,2021-05-14,11:33:43,UTC,1372495337760436228,Awadh460,Awadh,Kingdom of Saudi Arabia,RT @Live_Hindustan: #RubinaDilaik ने #Eid के म...,['Live_Hindustan'],"[{'url': 'https://t.co/B5L3ywz4bz', 'expanded_...",[],0,70,0,"['RubinaDilaik', 'Eid']",[],http://twitter.com/anyuser/status/139316763986...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",1.088526e+08,Fri May 14 10:09:54 +0000 2021,1393146544421961729
6,1393167634808537088,1393167634808537088,2021-05-14 11:33:42,2021-05-14,11:33:42,UTC,454609043,Amitkrkanhaiya,Amit Kumar Kanhaiya,Delhi,RT @ndtvindia: कोविड की राहत सामग्री को लेकर य...,['ndtvindia'],[],[],0,253,0,[],[],http://twitter.com/anyuser/status/139316763480...,False,,0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",3.212717e+08,Fri May 14 09:20:51 +0000 2021,1393134203663908864
7,1393167603049197574,1393167603049197574,2021-05-14 11:33:35,2021-05-14,11:33:35,UTC,959359483184115712,Bijendr90826388,Bijendra Singh,भारत,RT @thewirehindi: गोवा: शीर्ष सरकारी अस्पताल म...,['thewirehindi'],"[{'url': 'https://t.co/9irscDaR8V', 'expanded_...",[],0,7,0,['Covid19'],[],http://twitter.com/anyuser/status/139316760304...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",8.173406e+17,Fri May 14 11:12:03 +0000 2021,1393162185879265280
8,1393167590315364354,1393167590315364354,2021-05-14 11:33:31,2021-05-14,11:33:31,UTC,979348471521034241,DhootNavratan,Navratan Dhoot,"Rajasthan, India",RT @MyGovHindi: मास्क हमेशा सही तरीके से पहनें...,['MyGovHindi'],[]

In [ ]:
def calculate_days_past(tweet_time):
    #Calculating Dates
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    a1 = int(tweet_time[-1])
    b1 = int(months.index(tweet_time[1])+1)
    c1 = int(tweet_time[2])

    #Todays date
    today = date.today()
    life  =  str(date(today.year,today.month,today.day) - date(a1,b1,c1))
    if(len(life)>7):
        life  = life.split(" ")
        life  = life[0]
        life  = int(life)
        return life
    else:
        return 0

In [ ]:
def get_user_info(tweet):
    
    tweet_text   = tweet.full_text
    real_name    = tweet._json['user']['name']
    user_handle  = tweet._json['user']['screen_name']
    tweet_id     = tweet._json['id']
    user_id      = tweet._json['user']['id']
    
    user_info = api.get_user(user_id,user_mode='extended')
    desc = user_info._json['description']
    
    try:
        expanded_url = user_info._json['entities']['url']['urls'][0]['expanded_url']
        expanded_url = 1
    except:
        expanded_url = 0

    
    tweet_link             = "http://twitter.com/anyuser/status/" + str(tweet.id)
    chars_in_desc          = len(desc)
    chars_in_real_name     = len(real_name)
    chars_in_user_handle   = len(user_handle)
    num_matches            = len(difflib.SequenceMatcher(None, real_name,user_handle).get_matching_blocks())
    total_urls_in_desc     = len(user_info._json['entities']['description']['urls'])
    official_url_exists    = expanded_url
    followers_count        = user_info._json['followers_count']
    friends_count          = user_info._json['friends_count']
    listed_count           = user_info._json['listed_count']
    favourites_count       = user_info._json['favourites_count']
    geo_enabled            = int(user_info._json['geo_enabled'])
    acc_created_on         = user_info._json['created_at'].split()
    acc_life               = calculate_days_past(acc_created_on)
    verified               = int(user_info._json['verified'])
    num_tweets             = user_info._json['statuses_count']
    protected              = int(user_info._json['protected'])
    avg_likes_per_tweet    = favourites_count/num_tweets
    latest_tweet_time      = user_info._json['status']['created_at'].split()
    activity               = calculate_days_past(latest_tweet_time)

    if(acc_life==0):
        posting_frequency = num_tweets
    else:
        posting_frequency      = num_tweets/acc_life

    
    if(friends_count!=0):
        follower_friends_ratio = followers_count/friends_count
    else:
        follower_friends_ratio = 9999999
    
    list = [tweet_text, tweet_link, desc,
            real_name, user_handle, user_id,
            chars_in_desc, chars_in_real_name,
            chars_in_user_handle, num_matches,
            total_urls_in_desc, official_url_exists,
            followers_count, friends_count,
            listed_count, favourites_count,
            geo_enabled, acc_created_on,
            acc_life, verified, num_tweets,
            protected, posting_frequency,
            latest_tweet_time, activity,
            avg_likes_per_tweet, follower_friends_ratio]
    
    # for item in list:
    #     print(item)
        
    return list

In [ ]:
tweet_ids_list = df['id'].values.tolist()

In [ ]:
# Creating User Features Dataset
count = 1
new_users_features_list = []

for id in range(len(tweet_ids_list)):


    try:

        tweet = api.get_status(df['id'][id],tweet_mode='extended')
        user_info = get_user_info(tweet)
        user_info.insert(0,df['id'][id])
        new_users_features_list.append(user_info)
        count = count+1

        if(count>300):
            break

    except tweepy.TweepError as e:
        print("Tweepy error : " + str(e))
        continue

print(len(new_users_features_list))

27


In [ ]:
print(len(new_users_features_list))
print(count)

27
28


In [ ]:
df = pd.DataFrame(new_users_features_list,columns = [ 'tweet_id', 'tweet_text', 'tweet_link', 'desc', 'real_name', 'user_handle', 'user_id', 'chars_in_desc', 'chars_in_real_name',
                                                            'chars_in_user_handle', 'num_matches', 'total_urls_in_desc', 'official_url_exists', 'followers_count',
                                                            'friends_count', 'listed_count', 'favourites_count', 'geo_enabled', 'acc_created_on', 'acc_life', 'verified',
                                                            'num_tweets', 'protected', 'posting_frequency', 'latest_tweet_time', 'activity', 'avg_likes_per_tweet',
                                                            'follower_friends_ratio'])


In [ ]:
df

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio
0,1393167720020025345,"स्पूतनिक-वी वैक्सीन की कीमत का हो गया ऐलान, जा...",http://twitter.com/anyuser/status/139316772002...,"देश-दुनिया, अपने राज्य और शहर की सच्ची और अच्छ...",Oneindia Hindi,oneindiaHindi,18186549,99,14,13,3,1,1,63331,64,444,198,1,"[Wed, Dec, 17, 10:23:54, +0000, 2008]",4531,1,269712,0,59.525932,"[Fri, May, 14, 11:34:02, +0000, 2021]",0,0.000734,989.546875
1,1393167702051614725,RT @kmsingh1963: श्रीनिवास @srinivasiyc Covid-...,http://twitter.com/anyuser/status/139316770205...,"Engineer at telecommunication, Allahabad",Rajesh Yadav,RajeshY6762,925254155131748353,40,12,11,3,0,0,1598,35,0,176496,0,"[Tue, Oct, 31, 06:52:35, +0000, 2017]",1291,0,132069,0,102.299768,"[Fri, May, 14, 11:34:08, +0000, 2021]",0,1.336392,45.657143
2,1393167695437205507,RT @vinodkapri: मैं कभी कभी सोचता हूँ कि वो कौ...,http://twitter.com/anyuser/status/139316769543...,engineer..educator...civil service aspirant......,Sonu Bansal,bansalsonu95,812960111610236928,79,11,12,2,0,0,3,143,0,63,0,"[Sun, Dec, 25, 09:56:09, +0000, 2016]",1601,0,209,0,0.130543,"[Fri, May, 14, 11:33:57, +0000, 2021]",0,0.301435,0.020979
3,1393167680719310849,ab pata chala ki kyu @smritiirani ji aur @Mane...,http://twitter.com/anyuser/status/139316768071...,How can I describe myself ? Get to Know me first,Nilesh Jadav,Nileshj30966877,1271852296754864128,48,12,15,2,0,0,279,446,1,33931,0,"[Sat, Jun, 13, 17:10:13, +0000, 2020]",335,0,5601,0,16.719403,"[Fri, May, 14, 11:33:53, +0000, 2021]",0,6.058025,0.625561
4,1393167658908934146,RT @zeerajasthan_: कोरोना आपदा के कारण सारे त्...,http://twitter.com/anyuser/status/139316765890...,अपना हम क्या बताए दुनिया जानती है,Ganesh Neemrot,GNeemrot,1387475468266311680,33,14,8,3,0,0,0,13,0,0,0,"[Wed, Apr, 28, 18:36:14, +0000, 2021]",16,0,3,0,0.187500,"[Fri, May, 14, 11:33:48, +0000, 2021]",0,0.000000,0.000000
5,1393167639866925056,RT @Live_Hindustan: #RubinaDilaik ने #Eid के म...,http://twitter.com/anyuser/status/139316763986...,,Awadh,Awadh460,1372495337760436228,0,5,8,2,0,0,255,444,0,3388,0,"[Thu, Mar, 18, 10:30:17, +0000, 2021]",57,0,7251,0,127.210526,"[Fri, May, 14, 11:33:43, +0000, 2021]",0,0.467246,0.574324
6,1393167634808537088,RT @ndtvindia: कोविड की राहत सामग्री को लेकर य...,http://twitter.com/anyuser/status/139316763480...,"अगर रख सको तो एक निशानी हूँ मैं, खो दो तो सिर्...",Amit Kumar Kanhaiya,Amitkrkanhaiya,454609043,67,19,14,4,0,0,702,1124,4,7272,0,"[Wed, Jan, 04, 05:31:02, +0000, 2012]",3418,0,3808,0,1.114102,"[Fri, May, 14, 11:33:51, +0000, 2021]",0,1.909664,0.624555
7,1393167603049197574,RT @thewirehindi: गोवा: शीर्ष सरकारी अस्पताल म...,http://twitter.com/anyuser/status/139316760304...,अध्यक्ष प्रगतिशील जन आंदोलन,Bijendra Singh,Bijendr90826388,959359483184115712,27,14,15,2,0,0,238,57,0,41525,0,"[Fri, Feb, 02, 09:34:59, +0000, 2018]",1197,0,22235,0,18.575606,"[Fri, May, 14, 11:33:35, +0000, 2021]",0,1.867551,4.175439
8,1393167590315364354,RT @MyGovHindi: मास्क हमेशा सही तरीके से पहनें...,http://twitter.com/anyuser/status/139316759031...,Human first..Indian next..\nStudent\nFacebook:...,Navratan Dhoot,DhootNavratan,979348471521034241,136,14,13,2,3,0,11,123,0,921,0,"[Thu, Mar, 29, 13:24:05, +0000, 2018]",1142,0,413,0,0.361646,"[Fri, May, 14, 11:33:31, +0000, 2021]",0,2.230024,0.089431
9,1393167584917262339,RT @QuintHindi: कोरोना दवाएं बांटने को लेकर @s...,http://twitter.com/anyuser/status/139316758491...,"Social worker & Activist , Congres Sevadal &Al...",Brijesh Kumar Singh,Brijesh93565743,1104005837201137665,154,19,15,2,0,1,1328,1298,0,172947,0,"[Fri, Mar, 08, 13:07:57, +0000, 2019]",798,0,109784,0,137.573935,"[Fri, May, 14, 11:34:11, +0000, 2

In [ ]:
def preprocess_tweet(tweet):

    tweet = tweet.lower()
    url = r'http\S+'
    tweet = re.sub(url, 'URL', tweet, flags=re.MULTILINE)
    emoji = re.compile("["         u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002500-\U00002BEF"  # chinese char
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
    tweet =  emoji.sub(r'', tweet)
    tweet = ' '.join([word[1:] if word[0] == '#' else word for word in tweet.split()])
    
    return tweet

In [ ]:
pip install googletrans==3.1.0a0

In [ ]:
from googletrans import Translator
translator = Translator()

ModuleNotFoundError: ignored

In [ ]:
import re

for i in range(len(df)):
    df.at[i,'tweet_text'] = preprocess_tweet(df.iloc[i]['tweet_text'])

In [ ]:
def too_much_cleaned_tweet(tweet):

    
    cleaned_text = tweet.split()
    exclude_char_list = ['#','@','rt','|','%', '[', '(', '{', '}', ']', ')','url','URL','covid','covid19','corona','coronavirus','0','1','2','3','4','5','6','7','8','9']
    tweet_without_special_chars = [word for word in cleaned_text if all(ch not in word for ch in exclude_char_list)]
    tweet_without_special_chars = ' '.join(tweet_without_special_chars)
 
    return tweet_without_special_chars

    


In [ ]:
list = []
for tweet_text in df['tweet_text']:
    list.append(too_much_cleaned_tweet(tweet_text))

cleaned_tweet_df = pd.DataFrame(list,columns = ['too_much_cleaned_tweet'])
cleaned_tweet_df

,too_much_cleaned_tweet
0,"स्पूतनिक-वी वैक्सीन की कीमत का हो गया ऐलान, जा..."
1,श्रीनिवास की शुरुआत से ही सड़कों पर लोगों की स...
2,मैं कभी कभी सोचता हूँ कि वो कौन अफ़सर होता होग...
3,ab pata chala ki kyu ji aur ji mahamari me bhi...
4,कोरोना आपदा के कारण सारे त्यौहार अब सिर्फ नाम ...
5,rubinadilaik ने eid के मौके पर वीडियो शेयर कर ...
6,कोविड की राहत सामग्री को लेकर यूथ कांग्रेस के ...
7,गोवा: शीर्ष सरकारी अस्पताल में ऑक्सीजन का दबाव...
8,मास्क हमेशा सही तरीके से पहनें। इसके इस्तेमाल ...
9,"कोरोना दवाएं बांटने को लेकर से हुई पूछताछ, राह..."


In [ ]:
df = pd.concat([df,cleaned_tweet_df],axis=1)
df

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio,too_much_cleaned_tweet
0,1393167720020025345,"स्पूतनिक-वी वैक्सीन की कीमत का हो गया ऐलान, जा...",http://twitter.com/anyuser/status/139316772002...,"देश-दुनिया, अपने राज्य और शहर की सच्ची और अच्छ...",Oneindia Hindi,oneindiaHindi,18186549,99,14,13,3,1,1,63331,64,444,198,1,"[Wed, Dec, 17, 10:23:54, +0000, 2008]",4531,1,269712,0,59.525932,"[Fri, May, 14, 11:34:02, +0000, 2021]",0,0.000734,989.546875,"स्पूतनिक-वी वैक्सीन की कीमत का हो गया ऐलान, जा..."
1,1393167702051614725,rt @kmsingh1963: श्रीनिवास @srinivasiyc covid-...,http://twitter.com/anyuser/status/139316770205...,"Engineer at telecommunication, Allahabad",Rajesh Yadav,RajeshY6762,925254155131748353,40,12,11,3,0,0,1598,35,0,176496,0,"[Tue, Oct, 31, 06:52:35, +0000, 2017]",1291,0,132069,0,102.299768,"[Fri, May, 14, 11:34:08, +0000, 2021]",0,1.336392,45.657143,श्रीनिवास की शुरुआत से ही सड़कों पर लोगों की स...
2,1393167695437205507,rt @vinodkapri: मैं कभी कभी सोचता हूँ कि वो कौ...,http://twitter.com/anyuser/status/139316769543...,engineer..educator...civil service aspirant......,Sonu Bansal,bansalsonu95,812960111610236928,79,11,12,2,0,0,3,143,0,63,0,"[Sun, Dec, 25, 09:56:09, +0000, 2016]",1601,0,209,0,0.130543,"[Fri, May, 14, 11:33:57, +0000, 2021]",0,0.301435,0.020979,मैं कभी कभी सोचता हूँ कि वो कौन अफ़सर होता होग...
3,1393167680719310849,ab pata chala ki kyu @smritiirani ji aur @mane...,http://twitter.com/anyuser/status/139316768071...,How can I describe myself ? Get to Know me first,Nilesh Jadav,Nileshj30966877,1271852296754864128,48,12,15,2,0,0,279,446,1,33931,0,"[Sat, Jun, 13, 17:10:13, +0000, 2020]",335,0,5601,0,16.719403,"[Fri, May, 14, 11:33:53, +0000, 2021]",0,6.058025,0.625561,ab pata chala ki kyu ji aur ji mahamari me bhi...
4,1393167658908934146,rt @zeerajasthan_: कोरोना आपदा के कारण सारे त्...,http://twitter.com/anyuser/status/139316765890...,अपना हम क्या बताए दुनिया जानती है,Ganesh Neemrot,GNeemrot,1387475468266311680,33,14,8,3,0,0,0,13,0,0,0,"[Wed, Apr, 28, 18:36:14, +0000, 2021]",16,0,3,0,0.187500,"[Fri, May, 14, 11:33:48, +0000, 2021]",0,0.000000,0.000000,कोरोना आपदा के कारण सारे त्यौहार अब सिर्फ नाम ...
5,1393167639866925056,rt @live_hindustan: rubinadilaik ने eid के मौक...,http://twitter.com/anyuser/status/139316763986...,,Awadh,Awadh460,1372495337760436228,0,5,8,2,0,0,255,444,0,3388,0,"[Thu, Mar, 18, 10:30:17, +0000, 2021]",57,0,7251,0,127.210526,"[Fri, May, 14, 11:33:43, +0000, 2021]",0,0.467246,0.574324,rubinadilaik ने eid के मौके पर वीडियो शेयर कर ...
6,1393167634808537088,rt @ndtvindia: कोविड की राहत सामग्री को लेकर य...,http://twitter.com/anyuser/status/139316763480...,"अगर रख सको तो एक निशानी हूँ मैं, खो दो तो सिर्...",Amit Kumar Kanhaiya,Amitkrkanhaiya,454609043,67,19,14,4,0,0,702,1124,4,7272,0,"[Wed, Jan, 04, 05:31:02, +0000, 2012]",3418,0,3808,0,1.114102,"[Fri, May, 14, 11:33:51, +0000, 2021]",0,1.909664,0.624555,कोविड की राहत सामग्री को लेकर यूथ कांग्रेस के ...
7,1393167603049197574,rt @thewirehindi: गोवा: शीर्ष सरकारी अस्पताल म...,http://twitter.com/anyuser/status/139316760304...,अध्यक्ष प्रगतिशील जन आंदोलन,Bijendra Singh,Bijendr90826388,959359483184115712,27,14,15,2,0,0,238,57,0,41525,0,"[Fri, Feb, 02, 09:34:59, +0000, 2018]",1197,0,22235,0,18.575606,"[Fri, May, 14, 11:33:35, +0000, 2021]",0,1.867551,4.175439,गोवा: शीर्ष सरकारी अस्पताल में ऑक्सीजन का दबाव...
8,1393167590315364354,rt @mygovhindi: मास्क हमेशा सही तरीके से पहनें...,http://twitter.com/anyuser/status/139316759031...,Human first..Indian next..\nStudent\nFacebook:...,Navratan Dhoot,DhootNavratan,979348471521034241,136,14,13,2,3,0,11,123,0,921,0,"[Thu, Mar, 29, 13:24:05, +0000, 2018]",1142,0,413,0,0.361646,"[

In [ ]:
idx_drop = [index for index, row in df.iterrows() if translator.detect(row['too_much_cleaned_tweet']).lang == 'en']
idx_drop

[]

In [ ]:
df_modified = copy.deepcopy(df)
df_modified = df_modified.drop(idx_drop)
df_modified = df_modified.drop(['too_much_cleaned_tweet'],axis=1)
df_modified

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio
0,1393167720020025345,"स्पूतनिक-वी वैक्सीन की कीमत का हो गया ऐलान, जा...",http://twitter.com/anyuser/status/139316772002...,"देश-दुनिया, अपने राज्य और शहर की सच्ची और अच्छ...",Oneindia Hindi,oneindiaHindi,18186549,99,14,13,3,1,1,63331,64,444,198,1,"[Wed, Dec, 17, 10:23:54, +0000, 2008]",4531,1,269712,0,59.525932,"[Fri, May, 14, 11:34:02, +0000, 2021]",0,0.000734,989.546875
1,1393167702051614725,rt @kmsingh1963: श्रीनिवास @srinivasiyc covid-...,http://twitter.com/anyuser/status/139316770205...,"Engineer at telecommunication, Allahabad",Rajesh Yadav,RajeshY6762,925254155131748353,40,12,11,3,0,0,1598,35,0,176496,0,"[Tue, Oct, 31, 06:52:35, +0000, 2017]",1291,0,132069,0,102.299768,"[Fri, May, 14, 11:34:08, +0000, 2021]",0,1.336392,45.657143
2,1393167695437205507,rt @vinodkapri: मैं कभी कभी सोचता हूँ कि वो कौ...,http://twitter.com/anyuser/status/139316769543...,engineer..educator...civil service aspirant......,Sonu Bansal,bansalsonu95,812960111610236928,79,11,12,2,0,0,3,143,0,63,0,"[Sun, Dec, 25, 09:56:09, +0000, 2016]",1601,0,209,0,0.130543,"[Fri, May, 14, 11:33:57, +0000, 2021]",0,0.301435,0.020979
3,1393167680719310849,ab pata chala ki kyu @smritiirani ji aur @mane...,http://twitter.com/anyuser/status/139316768071...,How can I describe myself ? Get to Know me first,Nilesh Jadav,Nileshj30966877,1271852296754864128,48,12,15,2,0,0,279,446,1,33931,0,"[Sat, Jun, 13, 17:10:13, +0000, 2020]",335,0,5601,0,16.719403,"[Fri, May, 14, 11:33:53, +0000, 2021]",0,6.058025,0.625561
4,1393167658908934146,rt @zeerajasthan_: कोरोना आपदा के कारण सारे त्...,http://twitter.com/anyuser/status/139316765890...,अपना हम क्या बताए दुनिया जानती है,Ganesh Neemrot,GNeemrot,1387475468266311680,33,14,8,3,0,0,0,13,0,0,0,"[Wed, Apr, 28, 18:36:14, +0000, 2021]",16,0,3,0,0.187500,"[Fri, May, 14, 11:33:48, +0000, 2021]",0,0.000000,0.000000
5,1393167639866925056,rt @live_hindustan: rubinadilaik ने eid के मौक...,http://twitter.com/anyuser/status/139316763986...,,Awadh,Awadh460,1372495337760436228,0,5,8,2,0,0,255,444,0,3388,0,"[Thu, Mar, 18, 10:30:17, +0000, 2021]",57,0,7251,0,127.210526,"[Fri, May, 14, 11:33:43, +0000, 2021]",0,0.467246,0.574324
6,1393167634808537088,rt @ndtvindia: कोविड की राहत सामग्री को लेकर य...,http://twitter.com/anyuser/status/139316763480...,"अगर रख सको तो एक निशानी हूँ मैं, खो दो तो सिर्...",Amit Kumar Kanhaiya,Amitkrkanhaiya,454609043,67,19,14,4,0,0,702,1124,4,7272,0,"[Wed, Jan, 04, 05:31:02, +0000, 2012]",3418,0,3808,0,1.114102,"[Fri, May, 14, 11:33:51, +0000, 2021]",0,1.909664,0.624555
7,1393167603049197574,rt @thewirehindi: गोवा: शीर्ष सरकारी अस्पताल म...,http://twitter.com/anyuser/status/139316760304...,अध्यक्ष प्रगतिशील जन आंदोलन,Bijendra Singh,Bijendr90826388,959359483184115712,27,14,15,2,0,0,238,57,0,41525,0,"[Fri, Feb, 02, 09:34:59, +0000, 2018]",1197,0,22235,0,18.575606,"[Fri, May, 14, 11:33:35, +0000, 2021]",0,1.867551,4.175439
8,1393167590315364354,rt @mygovhindi: मास्क हमेशा सही तरीके से पहनें...,http://twitter.com/anyuser/status/139316759031...,Human first..Indian next..\nStudent\nFacebook:...,Navratan Dhoot,DhootNavratan,979348471521034241,136,14,13,2,3,0,11,123,0,921,0,"[Thu, Mar, 29, 13:24:05, +0000, 2018]",1142,0,413,0,0.361646,"[Fri, May, 14, 11:33:31, +0000, 2021]",0,2.230024,0.089431
9,1393167584917262339,rt @quinthindi: कोरोना दवाएं बांटने को लेकर @s...,http://twitter.com/anyuser/status/139316758491...,"Social worker & Activist , Congres Sevadal &Al...",Brijesh Kumar Singh,Brijesh93565743,1104005837201137665,154,19,15,2,0,1,1328,1298,0,172947,0,"[Fri, Mar, 08, 13:07:57, +0000, 2019]",798,0,109784,0,137.573935,"[Fri, May, 14, 11:34:11, +0000, 2

In [ ]:
### Adding Label's field --> (( 1=Real || 2=Fake || 3=Maybe ))

df['label'] = pd.DataFrame((np.ones(len(df),dtype=int)*3))
df

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio,too_much_cleaned_tweet,label
0,1393167720020025345,"स्पूतनिक-वी वैक्सीन की कीमत का हो गया ऐलान, जा...",http://twitter.com/anyuser/status/139316772002...,"देश-दुनिया, अपने राज्य और शहर की सच्ची और अच्छ...",Oneindia Hindi,oneindiaHindi,18186549,99,14,13,3,1,1,63331,64,444,198,1,"[Wed, Dec, 17, 10:23:54, +0000, 2008]",4531,1,269712,0,59.525932,"[Fri, May, 14, 11:34:02, +0000, 2021]",0,0.000734,989.546875,"स्पूतनिक-वी वैक्सीन की कीमत का हो गया ऐलान, जा...",3
1,1393167702051614725,rt @kmsingh1963: श्रीनिवास @srinivasiyc covid-...,http://twitter.com/anyuser/status/139316770205...,"Engineer at telecommunication, Allahabad",Rajesh Yadav,RajeshY6762,925254155131748353,40,12,11,3,0,0,1598,35,0,176496,0,"[Tue, Oct, 31, 06:52:35, +0000, 2017]",1291,0,132069,0,102.299768,"[Fri, May, 14, 11:34:08, +0000, 2021]",0,1.336392,45.657143,श्रीनिवास की शुरुआत से ही सड़कों पर लोगों की स...,3
2,1393167695437205507,rt @vinodkapri: मैं कभी कभी सोचता हूँ कि वो कौ...,http://twitter.com/anyuser/status/139316769543...,engineer..educator...civil service aspirant......,Sonu Bansal,bansalsonu95,812960111610236928,79,11,12,2,0,0,3,143,0,63,0,"[Sun, Dec, 25, 09:56:09, +0000, 2016]",1601,0,209,0,0.130543,"[Fri, May, 14, 11:33:57, +0000, 2021]",0,0.301435,0.020979,मैं कभी कभी सोचता हूँ कि वो कौन अफ़सर होता होग...,3
3,1393167680719310849,ab pata chala ki kyu @smritiirani ji aur @mane...,http://twitter.com/anyuser/status/139316768071...,How can I describe myself ? Get to Know me first,Nilesh Jadav,Nileshj30966877,1271852296754864128,48,12,15,2,0,0,279,446,1,33931,0,"[Sat, Jun, 13, 17:10:13, +0000, 2020]",335,0,5601,0,16.719403,"[Fri, May, 14, 11:33:53, +0000, 2021]",0,6.058025,0.625561,ab pata chala ki kyu ji aur ji mahamari me bhi...,3
4,1393167658908934146,rt @zeerajasthan_: कोरोना आपदा के कारण सारे त्...,http://twitter.com/anyuser/status/139316765890...,अपना हम क्या बताए दुनिया जानती है,Ganesh Neemrot,GNeemrot,1387475468266311680,33,14,8,3,0,0,0,13,0,0,0,"[Wed, Apr, 28, 18:36:14, +0000, 2021]",16,0,3,0,0.187500,"[Fri, May, 14, 11:33:48, +0000, 2021]",0,0.000000,0.000000,कोरोना आपदा के कारण सारे त्यौहार अब सिर्फ नाम ...,3
5,1393167639866925056,rt @live_hindustan: rubinadilaik ने eid के मौक...,http://twitter.com/anyuser/status/139316763986...,,Awadh,Awadh460,1372495337760436228,0,5,8,2,0,0,255,444,0,3388,0,"[Thu, Mar, 18, 10:30:17, +0000, 2021]",57,0,7251,0,127.210526,"[Fri, May, 14, 11:33:43, +0000, 2021]",0,0.467246,0.574324,rubinadilaik ने eid के मौके पर वीडियो शेयर कर ...,3
6,1393167634808537088,rt @ndtvindia: कोविड की राहत सामग्री को लेकर य...,http://twitter.com/anyuser/status/139316763480...,"अगर रख सको तो एक निशानी हूँ मैं, खो दो तो सिर्...",Amit Kumar Kanhaiya,Amitkrkanhaiya,454609043,67,19,14,4,0,0,702,1124,4,7272,0,"[Wed, Jan, 04, 05:31:02, +0000, 2012]",3418,0,3808,0,1.114102,"[Fri, May, 14, 11:33:51, +0000, 2021]",0,1.909664,0.624555,कोविड की राहत सामग्री को लेकर यूथ कांग्रेस के ...,3
7,1393167603049197574,rt @thewirehindi: गोवा: शीर्ष सरकारी अस्पताल म...,http://twitter.com/anyuser/status/139316760304...,अध्यक्ष प्रगतिशील जन आंदोलन,Bijendra Singh,Bijendr90826388,959359483184115712,27,14,15,2,0,0,238,57,0,41525,0,"[Fri, Feb, 02, 09:34:59, +0000, 2018]",1197,0,22235,0,18.575606,"[Fri, May, 14, 11:33:35, +0000, 2021]",0,1.867551,4.175439,गोवा: शीर्ष सरकारी अस्पताल में ऑक्सीजन का दबाव...,3
8,1393167590315364354,rt @mygovhindi: मास्क हमेशा सही तरीके से पहनें...,http://twitter.com/anyuser/status/139316759031...,Human first..Indian next..\nStudent\nFacebook:...,Navratan Dhoot,DhootNavratan,979348471521034241,136,14,13,2,3,0,11,123,0,921,0,"[Thu, Mar, 29, 13:24:05, +0000, 2018]",11

In [ ]:
df.to_csv("/gdrive/My Drive/Major/Dataset/Scrapping tweets dem/covid_tweets_with_user_features.csv",index=False)